In [1]:
import pandas as pd
import sys
import os 

# sys.path.append("../")
# sys.path.append("../../")
# sys.path.append("../../../")



In [1]:
from dotenv import dotenv_values

config = dotenv_values(".env.dev")

config["KREAM_DETAILS_TEMP_DIR"] 

'router/dev/kream/data/detail/temp/'

In [1]:
from router.dev.kream import create_last_update_kream_detail_log,get_kream_product_card

data  =create_last_update_kream_detail_log("231103-012830-adidas")

231103-012830 adidas


In [1]:
from router.dev.kream import get_kream_buy_and_sell
x = get_kream_buy_and_sell('adidas','231103-012830',sample=20)

In [10]:
import json
json.dumps(x,default=str)

'{"len": 140, "kream_id_list": [96172, 96172, 96172, 96172, 96172, 96172, 96172, 96172, 96172, 96172, 96172, 96172, 96172, 96172, 96172, 96172, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 26344, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 64283, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 55267, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 122782, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 55955, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 90918, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251, 15251,